<a href="https://colab.research.google.com/github/wakepon21/Masa/blob/master/signate1_23.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Dataset

In [364]:
# LightGBM
import lightgbm as lgb
from sklearn.metrics import roc_auc_score

# Imbalanced-learn
from imblearn.under_sampling import RandomUnderSampler


In [365]:
#control
!pip install git+https://github.com/pandas-profiling/pandas-profiling.git
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
import pandas_profiling
import seaborn as sns
from pandas_profiling.utils.cache import cache_file

#Optuna
!pip install optuna
import optuna
from sklearn.metrics import log_loss

#何で必要なのか分かってない
%matplotlib inline
warnings.filterwarnings('ignore')

  Cloning https://github.com/pandas-profiling/pandas-profiling.git to /tmp/pip-req-build-y3ksobz4
  Running command git clone -q https://github.com/pandas-profiling/pandas-profiling.git /tmp/pip-req-build-y3ksobz4
  Created wheel for pandas-profiling: filename=pandas_profiling-2.9.0rc1-py2.py3-none-any.whl size=258106 sha256=82048978d18484f223c739f315dab94c8c06cba7e74c9153bf3a3e1d2d8d3abc
  Stored in directory: /tmp/pip-ephem-wheel-cache-t0yw378n/wheels/cd/13/75/8fcecd52c706914e90d916ede57f8c74de60e4c9ebc6c2f3b7
Successfully built pandas-profiling


In [366]:
#train,test,submit_sampleのみっつがそろっているか確認
!ls

sample_data		       submit_sample.csv  train.csv
submission_lightgbm_kfold.csv  test.csv


In [367]:
#そろっていなかったら選択
from google.colab import files
train_up = files.upload()

In [368]:
sub = pd.read_csv("submit_sample.csv",names=("A","B"))
test = pd.read_csv("test.csv")
train = pd.read_csv("train.csv")
data = pd.concat([train,test], sort=False)

#特徴量をエンジニアリング
data["income"]=data["job"].copy()
data["goodincome"]=data["job"].copy()
data["job"].replace(['blue-collar','management','technician','admin.','services',\
                     'unknown','self-employed','entrepreneur','student','retired',\
                     'unemployed','housemaid'],[0,1,2,3,4,5,6,7,8,9,10,11],inplace=True)
data["loan"].replace(['yes','no'],[1,0],inplace=True)
data["married"]=data["marital"]
data["single"]=data["marital"]
data['education'].replace(['secondary','tertiary','primary','unknown'],[3,1,0,2],inplace=True)
data["housing"].replace(['yes','no'],[1,0],inplace=True)
data["contact"].replace(['cellular','telephone','unknown'],[2,1,0],inplace=True)
data["poutcome"].replace(['success','failure','unknown','other'],[3,1,2,0],inplace=True)
#data["job"] = data["job"].astype(np.int64)

data["month"].replace(['jan','feb','mar','apr','may','jun','jul','aug','sep','oct','nov','des'],\
                      [3,4,5,6,7,8,9,10,11,0,1,2],inplace=True)
#data["y"].astype(np.int64)
#ここから追加
data["year"]=data["month"]*30+data["day"]
data["campaign+previous"]=data["campaign"] + data["previous"]
data["poutcome/pdays"]=data["poutcome"] / (data["pdays"]+10) * 10000
data["poutcome/pdays"] = data["poutcome/pdays"].astype(np.int64)
data["income"].replace(['blue-collar','management','technician',\
                                         'admin.','services','unknown','self-employed',\
                                         'entrepreneur','student','retired','unemployed','housemaid'],\
                                        [0,0,0,0,0,0,0,0,1,1,1,0],inplace=True)
data["goodincome"].replace(['blue-collar','management','technician','admin.','services','unknown','self-employed','entrepreneur','student','retired','unemployed','housemaid'],\
                                                [0,1,0,1,0,0,1,1,0,0,0,0],inplace=True)
data["housing+loan"] = data["housing"] + data["loan"]
data["duration*campaign"] = data["duration"] * data["campaign"] 
data["married"].replace(['married','single','divorced'],[1,0,1],inplace=True)
data["single"].replace(['married','single','divorced'],[1,0,0],inplace=True)
train["y"]=train["y"].astype(np.int64)

#全体のうち、残したいモノだけ選ぶ
all_columns = list(train.columns)
remain_columns = ['age','job','education','balance','housing','loan',
                  'contact','day','month','duration','campaign','pdays','previous',
                  'poutcome','y','income','goodincome',"married","single","year",
                  'campaign+previous'	,'poutcome/pdays'	,'housing+loan','duration*campaign']
                  
delete_columns = list(set(all_columns)-set(remain_columns))
data.drop(delete_columns, axis=1, inplace=True)

#trainとtestを再度切り分け
train = data[:len(train)]
test = data[len(train):]

#train,testを、さらに説明変数Xと、予測変数yに切り分け。y_testは与えられていないのでなし。三種類がでる
y = train['y']
X = train.drop('y', axis = 1)
X_sub = test.drop('y', axis = 1)
#Xcolumns=X_train.columns

In [369]:
def imbalanced_data_split(X, y, test_size=0.2):
    sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=0)
    for train_index, test_index in sss.split(X, y):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        return X_train, X_test, y_train, y_test

In [370]:
X_train, X_test, y_train, y_test = imbalanced_data_split(X.values, y.values, test_size=0.2)
X_train2, X_valid, y_train2, y_valid = imbalanced_data_split(X_train, y_train, test_size=0.2)

# LightGBM

In [371]:
def lgbm_train(X_train_df, X_valid_df, y_train_df, y_valid_df, lgbm_params):
    lgb_train = lgb.Dataset(X_train_df, y_train_df)
    lgb_eval = lgb.Dataset(X_valid_df, y_valid_df, reference=lgb_train)

    # 上記のパラメータでモデルを学習する
    model = lgb.train(lgbm_params, lgb_train,
                      # モデルの評価用データを渡す
                      valid_sets=lgb_eval,
                      # 最大で 1000 ラウンドまで学習する
                      num_boost_round=1000,
                      # 10 ラウンド経過しても性能が向上しないときは学習を打ち切る
                      early_stopping_rounds=10)
    
    return model

# Optuna

In [372]:
def objective(trial):
    params = {
        'objective': 'binary',
        'max_bin': trial.suggest_int('max_bin', 255, 500),
        'learning_rate': 0.1,
        'num_leaves': trial.suggest_int('num_leaves', 32, 128),
    }

    lgb_train = lgb.Dataset(X_train, y_train)
    lgb_eval = lgb.Dataset(X_valid, y_valid, reference=lgb_train)

    model = lgb.train(params, lgb_train,
                                   valid_sets=[lgb_train, lgb_eval],
                                   verbose_eval=10,
                                   num_boost_round=1000,
                                   early_stopping_rounds=10)

    y_pred_valid = model.predict(X_valid, num_iteration=model.best_iteration)
    score = log_loss(y_valid, y_pred_valid)
    return score

In [373]:
study = optuna.create_study(sampler=optuna.samplers.RandomSampler(seed=0))
study.optimize(objective, n_trials=2)

Training until validation scores don't improve for 10 rounds.
[10]	training's binary_logloss: 0.194557	valid_1's binary_logloss: 0.190526
[20]	training's binary_logloss: 0.168532	valid_1's binary_logloss: 0.164493
[30]	training's binary_logloss: 0.150763	valid_1's binary_logloss: 0.146542
[40]	training's binary_logloss: 0.135896	valid_1's binary_logloss: 0.132077
[50]	training's binary_logloss: 0.123953	valid_1's binary_logloss: 0.12111
[60]	training's binary_logloss: 0.114119	valid_1's binary_logloss: 0.111564
[70]	training's binary_logloss: 0.105829	valid_1's binary_logloss: 0.103601
[80]	training's binary_logloss: 0.0984376	valid_1's binary_logloss: 0.0963286
[90]	training's binary_logloss: 0.0914537	valid_1's binary_logloss: 0.0889243
[100]	training's binary_logloss: 0.0852107	valid_1's binary_logloss: 0.0828954
[110]	training's binary_logloss: 0.0792592	valid_1's binary_logloss: 0.0767909
[120]	training's binary_logloss: 0.0740323	valid_1's binary_logloss: 0.0717346
[130]	training

[I 2020-08-23 04:27:03,590] Trial 0 finished with value: 0.00032918029097301336 and parameters: {'max_bin': 427, 'num_leaves': 79}. Best is trial 0 with value: 0.00032918029097301336.


Training until validation scores don't improve for 10 rounds.
[10]	training's binary_logloss: 0.189629	valid_1's binary_logloss: 0.18571
[20]	training's binary_logloss: 0.161884	valid_1's binary_logloss: 0.157597
[30]	training's binary_logloss: 0.141819	valid_1's binary_logloss: 0.138057
[40]	training's binary_logloss: 0.125536	valid_1's binary_logloss: 0.121
[50]	training's binary_logloss: 0.112451	valid_1's binary_logloss: 0.108623
[60]	training's binary_logloss: 0.102097	valid_1's binary_logloss: 0.0987344
[70]	training's binary_logloss: 0.0934972	valid_1's binary_logloss: 0.0906934
[80]	training's binary_logloss: 0.0863886	valid_1's binary_logloss: 0.0841452
[90]	training's binary_logloss: 0.0791508	valid_1's binary_logloss: 0.0769965
[100]	training's binary_logloss: 0.072249	valid_1's binary_logloss: 0.070104
[110]	training's binary_logloss: 0.066403	valid_1's binary_logloss: 0.0643689
[120]	training's binary_logloss: 0.0612806	valid_1's binary_logloss: 0.0590974
[130]	training's 

[I 2020-08-23 04:27:14,012] Trial 1 finished with value: 7.468242595877394e-05 and parameters: {'max_bin': 372, 'num_leaves': 96}. Best is trial 1 with value: 7.468242595877394e-05.


In [374]:
study.best_params

{'max_bin': 372, 'num_leaves': 96}

In [375]:
lgbm_params = {
    'learning_rate': 0.05,
    'num_leaves': study.best_params["num_leaves"],
    'boosting_type' : 'gbdt',
    'reg_alpha' : 1,
    'reg_lambda' : 1,
    'objective': 'binary',
    'max_bin': study.best_params["max_bin"],
    'metric': 'auc',
    'max_dapth': 6
}

#categorical_features = ['job','education','housing','loan','contact','poutcome',"income",'goodincome','married',"single",'housing+loan']
#X_train[categorical_features]=X_train[categorical_features].astype('category')

# Bagging

In [376]:
def bagging(seed):
    sampler = RandomUnderSampler(random_state=seed, replacement=True)
    X_resampled, y_resampled = sampler.fit_resample(X_train, y_train)
    X_train2, X_valid, y_train2, y_valid = imbalanced_data_split(X_resampled, y_resampled, test_size=0.2)
    model_bagging = lgbm_train(X_train2, X_valid, y_train2, y_valid, lgbm_params)
    return model_bagging

In [377]:
%%time
models = []

for i in range(10):
    models.append(bagging(i))

[1]	valid_0's auc: 0.797342
Training until validation scores don't improve for 10 rounds.
[2]	valid_0's auc: 0.796309
[3]	valid_0's auc: 0.795805
[4]	valid_0's auc: 0.795709
[5]	valid_0's auc: 0.795937
[6]	valid_0's auc: 0.794926
[7]	valid_0's auc: 0.802344
[8]	valid_0's auc: 0.807403
[9]	valid_0's auc: 0.806178
[10]	valid_0's auc: 0.807609
[11]	valid_0's auc: 0.80873
[12]	valid_0's auc: 0.808445
[13]	valid_0's auc: 0.809767
[14]	valid_0's auc: 0.81027
[15]	valid_0's auc: 0.809802
[16]	valid_0's auc: 0.809973
[17]	valid_0's auc: 0.808887
[18]	valid_0's auc: 0.81045
[19]	valid_0's auc: 0.811124
[20]	valid_0's auc: 0.811224
[21]	valid_0's auc: 0.810857
[22]	valid_0's auc: 0.810826
[23]	valid_0's auc: 0.810695
[24]	valid_0's auc: 0.810275
[25]	valid_0's auc: 0.811421
[26]	valid_0's auc: 0.812822
[27]	valid_0's auc: 0.813246
[28]	valid_0's auc: 0.814192
[29]	valid_0's auc: 0.814918
[30]	valid_0's auc: 0.815159
[31]	valid_0's auc: 0.814901
[32]	valid_0's auc: 0.814927
[33]	valid_0's auc: 0.

In [378]:
y_preds = []

for m in models:
    y_preds.append(m.predict(X_test, num_iteration=m.best_iteration))

y_preds_bagging = sum(y_preds)/len(y_preds)
# auc を計算する
auc = roc_auc_score(y_test, y_preds_bagging)
print(auc)

for y_pred in y_preds:
    print(roc_auc_score(y_test, y_pred))

0.8435516386649446
0.8331967858233341
0.8392058245098988
0.8294467075929898
0.8219662272491935
0.8214257266887799
0.8297778827170679
0.8289673689380965
0.8169628040789775
0.8222675326054593
0.8321591669843293


In [379]:
y_subs = []

for m in models:
    y_subs.append(m.predict(X_sub, num_iteration=m.best_iteration))

y_subs_bagging = sum(y_subs)/len(y_subs)
# auc を計算する
y_subs_bagging

array([0.9176974 , 0.26843237, 0.1937739 , ..., 0.53272628, 0.08965956,
       0.63559955])

In [380]:
percentage=0.37
y_sub = (y_subs_bagging > percentage).astype(int)

sub['B'] = y_sub
sub.to_csv('submission_lightgbm_kfold.csv', index=False, header=False)

sum(sub["B"]),len(sub["B"])

(8619, 18050)

In [381]:
files.download('submission_lightgbm_kfold.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>